# Report if residues interacting with a specific chain have equivalent residues in an hhsuite-generated alignment

This notebook is meant to be used when you are studing a complex that contains at least two or more proteins because it is meant to generate a report on the residues of a protein chain of interest that interacts with another in a structure and tell you what parts of the chain of interest have equivalents in a remote homolog. This notebook illustrates bringing together the structural data from a released structure with a remote homology alignment by tools from HH-Suite3. The idea is you may have a protein where the structure has been solved and if that chain has been used to identify remote homologs, you may be curious about whether residues involved in interacting with other chains in the complex are found in the identified homolog and what are the identities.

This effort uses information from the PDBsum report for the corresponding chains in a specified structure. See [pdbsum-binder: demonstrating analysis of PDBsum-related data via active Jupyter sessions provided via MyBinder.org](https://github.com/fomightez/pdbsum-binder) and [my collection of utility scripts for working with data from PDBsum](https://github.com/fomightez/structurework/tree/master/pdbsum-utilities) for more resources abut PDBsum and using it in conjuction with Jupyter/Python.

The portion dealing with getting the protein-protein interaction data, and some of the general apprach to this example applying to one protein protein interaction as the initial example, is based on my notebook [Using PDBsum data to highlight changes in protein-protein interactions.ipynb](https://nbviewer.jupyter.org/github/fomightez/pdbsum-binder/blob/main/notebooks/Using%20PDBsum%20data%20to%20highlight%20changes%20in%20protein-protein%20interactions.ipynb) that is available and ready to be run in active Binder sessions launched from [here](https://github.com/fomightez/pdbsum-binder).  The idea is to expand on that for when you want to look at multiple structure examples for the involved chain and that follow-up will be based on [Using snakemake to highlight changes in multiple protein-protein interactions via PDBsum data.ipynb](https://nbviewer.jupyter.org/github/fomightez/pdbsum-binder/blob/main/notebooks/Using%20snakemake%20to%20highlight%20changes%20in%20multiple%20protein-protein%20interactions%20via%20PDBsum%20data.ipynb) available as an active running file in Binder sessions launched from [the same place](https://github.com/fomightez/pdbsum-binder).

----

## Preparation

#### Get necessary scripts

Get the `hhsuite3_results_to_df.py` script that will be used to mine the results file(s) generated by HH-suite3.

In [1]:
import os
file_needed = "hhsuite3_results_to_df.py"
if not os.path.isfile(file_needed):
    !curl -OL https://raw.githubusercontent.com/fomightez/sequencework/master/hhsuite3-utilities/hhsuite3_results_to_df.py

Get the `pdbsum_prot_interactions_list_to_df.py` script that will be used to collect interaction data for a protein pair from [PDBsum](http://www.ebi.ac.uk/thornton-srv/databases/cgi-bin/pdbsum/GetPage.pl?pdbcode=index.html).

In [2]:
import os
file_needed = "pdbsum_prot_interactions_list_to_df.py"
if not os.path.isfile(file_needed):
    !curl -OL https://raw.githubusercontent.com/fomightez/structurework/master/pdbsum-utilities/pdbsum_prot_interactions_list_to_df.py

Get the `make_report_on_equivalents_of_interacting_residues.py` script that will be used to make the report on the equivalent residues in the structure that interact with other chains and the aligned sequence of the idenitified homolog.

In [3]:
import os
file_needed = "make_report_on_equivalents_of_interacting_residues.py"
if not os.path.isfile(file_needed):
    !curl -OL https://raw.githubusercontent.com/fomightez/sequencework/master/hhsuite3-utilities/make_report_on_equivalents_of_interacting_residues.py

#### Get example input data

Get an example file  HH-suite3-generated results.  

(You can use your own `.hhr` file if you have one and the chain used in the query has a corresponding structure; however, you'll need to change the entries in this demo notebook to correspond.)

For ease, I'll just some of the results files that the biopython package uses to test the parsing code is working well. They are publically available [here](https://github.com/biopython/biopython/tree/master/Tests/HHsuite); the `README` file [there](https://github.com/biopython/biopython/tree/master/Tests/HHsuite) summarizes the content of those files.

If you prefer to use your own result files, upload your file to this session and change the appropriate file names in later steps.  
(Note for a couple of the `.hhr` files, I noticed they had `Done!` close to the last line of the file; however, I hadn't seen that in the `.hhr` files from HH-suite3 that I developed with and it caused errors for my `hhsuite3_results_to_df.py`. To make them more consistent, I'm removing that, assuming it is from format version of HH-suite since biopython which is a source of these test files mentions that version in the current documentation.)

In [4]:
import os
file_needed = "results_S288C_NOP1.hhr"
if not os.path.isfile(file_needed):
    !curl -OL https://gist.githubusercontent.com/fomightez/cbdca72f5990146170f6d15789234dfb/raw/92f6ad30056b4c930d480b6da69265b0101e0cc6/results_S288C_NOP1.hhr

We'll see how to specify use of this later. You'd obviously want to replace use of this with your own HH-suite3-generated results to adapt this notebook to your own interests.

## Specifying particulars for the main analysis

### Getting the interactions file

The portion dealing with getting the protein-protein interaction data, and some of the general apprach to this example applying to one protein protein interaction as the initial example, is based on my notebook [Using PDBsum data to highlight changes in protein-protein interactions.ipynb](https://nbviewer.jupyter.org/github/fomightez/pdbsum-binder/blob/main/notebooks/Using%20PDBsum%20data%20to%20highlight%20changes%20in%20protein-protein%20interactions.ipynb) that is available and ready to be run in active Binder sessions launched from [here](https://github.com/fomightez/pdbsum-binder). 

Let's define the structure to use. (We'll use [6zdt](https://www.rcsb.org/structure/6ZDT) for the demo.)

In [5]:
structure = "6zdt"

Let's define the chains to use

In [6]:
structure_chain1 = "A"
structure_chain2 = "B"

Chain `A` in this structure is baker's yeast Nop1p, a.k.a. fibrillarin in other eukaryotic organisms.
Importantly, **chain1 here will be the one we need to correspond to the chain represented as the query chain the HH-suite-generated data.**  
If chain `B` in the structure had been the one used in the query and the other chain we wanted to examine it's interactions with was `X`, we would have made the code like this:

```python
structure_chain1 = "B"
structure_chain2 = "X"
```

Next, let's define what we want the data files saved as:

In [7]:
structure_chains_data_name = "datai_6zdt_A_B.txt"

Code based on first notebook in the pdbsum-binder launches to get the interaction data files for each protein pair for each of the two structures:

In [8]:
def get_protein_inter_data_files(pdb_code,chain1,chain2,output_file_name):
    '''
    Takes a PDB entry accession identifier alphanumeic (PDB code), a chain 
    identifier for chain 1 and chain identifier for chain 2, along with a
    name to give the file produced when the data is retrieved and saved.

    The proteins have to interact in the structure for meaningful data to be returned.
    '''
    source_url = "http://www.ebi.ac.uk/thornton-srv/databases/cgi-bin/pdbsum/GetIface.pl"
    !curl -L -o {output_file_name} --data "pdb={pdb_code.lower()}&chain1={chain1}&chain2={chain2}" {source_url}
# Get data file for the structure
get_protein_inter_data_files(structure,structure_chain1,structure_chain2,structure_chains_data_name)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16699    0 16673  100    26  15241     23  0:00:01  0:00:01 --:--:-- 15278


We can list the files present to verify the data file was obtained.

In [9]:
ls

 datai_6zdt_A_B.txt
 Encephalitozoon_cuniculi_ecuniii_l_gca_001078035.ECIIIL.pep.all.fa.gz
 hhs3_results_pickled_df.pkl
 hhsuite3_results_to_df.py
 make_report_on_equivalents_of_interacting_residues.py
 pdbsum_prot_interactions_list_to_df.py
 prot_int_pickled_df.pkl
 __pycache__/
'Report if residues interacting with a specific chain have equivalent residues in an hhsuite-generated alignment.ipynb'
 results_S288C_NOP1.hhr
'Using Python to examine HH-suite3 results.ipynb'*
'Using Python to examine HH-suite results - example making multi-fasta sequence format from query and hit sequences.ipynb'


`datai_6zdt_A_B.txt` will show in the list if all is okay.

Next we want to read in the data to make a useful dataframe from it using the main function of the script  `pdbsum_prot_interactions_list_to_df.py`that we retrieved earlier. (See 'Making a Pandas dataframe from the interactions file directly in Jupyter' in `Working with PDBsum in Jupyter Basics.ipynb` that is available and ready to be run in active Binder sessions launched from [here](https://github.com/fomightez/pdbsum-binder) to follow what is going on in this step. That notebook is available [here](https://nbviewer.jupyter.org/github/fomightez/pdbsum-binder/blob/main/notebooks/Working%20with%20PDBsum%20in%20Jupyter%20Basics.ipynb#Making-a-Pandas-dataframe-from-the-interactions-file-directly-in-Jupyter) in static form from [the pdbsum-binder repo](https://github.com/fomightez/pdbsum-binder).)

In [10]:
from pdbsum_prot_interactions_list_to_df import pdbsum_prot_interactions_list_to_df
i_df = pdbsum_prot_interactions_list_to_df("datai_6zdt_A_B.txt")

Provided interactions data read and converted to a dataframe...

A dataframe of the data has been saved as a file
in a manner where other Python programs can access it (pickled form).
RESULTING DATAFRAME is stored as ==> 'prot_int_pickled_df.pkl'

Returning a dataframe with the information as well.

We can look at the initial lines of data in the dataframe to make sure that worked by running the next cell:

In [11]:
i_df.head()

,Atom1 no.,Atom1 name,Atom1 Res name,Atom1 Res no.,Atom1 Chain,Atom2 no.,Atom2 name,Atom2 Res name,Atom2 Res no.,Atom2 Chain,Distance,type
0,1219,NZ,LYS,169,A,6012,OE1,GLU,148,B,2.97,Hydrogen bonds
1,1601,OH,TYR,195,A,6061,OE1,GLN,151,B,3.08,Hydrogen bonds
2,1760,NE,ARG,205,A,6266,O,VAL,163,B,3.31,Hydrogen bonds
3,1763,NH2,ARG,205,A,6266,O,VAL,163,B,3.05,Hydrogen bonds
4,2038,O,PRO,219,A,6176,OG,SER,159,B,2.71,Hydrogen bonds


### Getting the alignment of the yeast chain with the ortholog chain

This will be where we get the alignment of the chain represented in the structure with the ortholog in the other organism by extracting the alignment data from the HH-suite3-generated data.

We retrieved the results file `results_S288C_NOP1.hhr` earlier. And so we can easily make a dataframe of that using the main function of the script `hhsuite3_results_to_df` we retrieved earlier.  (See the bottom of the notebook `Using Python to examine HH-suite3 results.ipynb` that should be available in this session [or repo] if you need to better understand what is going on in this step.)

In [12]:
from hhsuite3_results_to_df import hhsuite3_results_to_df
hh_df = hhsuite3_results_to_df("results_S288C_NOP1.hhr")

Provided results read and converted to a dataframe...

A dataframe of the data has been saved as a file
in a manner where other Python programs can access it (pickled form).
RESULTING DATAFRAME is stored as ==> 'hhs3_results_pickled_df.pkl'

Returning a dataframe with the information as well.

We can look at the initial lines of data in the dataframe to make sure that worked by running the next cell:

In [13]:
hh_df.head()

,hit_num,qid,qtitle,query_length,qstart,qend,hid,htitle,hit_length,hstart,...,Similarity,Sum_probs,Template_Neff,size_diff,qseq,hconsensus,qconsensus,hseq,aln_confidence,pw_conservation
0,1,NOP1,NOP1 YDL014W,327,80,326,KMV65234,KMV65234 pep supercontig:ECIIIL:CH10_ECIII-L:9...,291,41,...,1.074,204.2,8.7,36,ARGGAKVVIEPH-RHAGVYIARGKEDLLVTKNMAPGESVYGEKRIS...,~~~~~~~~~~~~~~~~~~~~~~~~~~~~~t~~~~~g~~vyge~~~~...,~~~~~~~~~~~~-~~~~~~~~~~~~~~~~t~~~~~g~~vyge~~~~...,AGLDRKVLVEPHPRFPGVYISRGKEDLLLTRNLVPGVSVYGEKRVA...,4567899999999998888899999999999999999988753356...,.++++++.|+||++.+.+.++.+|||+|||+|.++..++...++.....
1,2,NOP1,NOP1 YDL014W,327,165,272,KMV66152,KMV66152 pep supercontig:ECIIIL:CH05_ECIII-L:1...,247,46,...,0.085,69.5,9.1,80,APGKKVLYLGAASGTSVSHVSDVVGPEGVVYAVEFSHRPGRELISM...,~~~~~vLDiGcG~G~~~~~l~~~---g~~v~gvD~s~~~i~~a~~~...,~~~~~VLDiG~G~G~~~~~la~~~~~~~~v~gvD~s~~~i~~a~~~...,KDGGLVLDVGCGSGLSGSVLSES---GYPWIGVDISMEMLKLGMER...,45678999999999998888765432,.++.+|||+|||+|.++..++...+.
2,3,NOP1,NOP1 YDL014W,327,165,272,KMV65418,KMV65418 pep supercontig:ECIIIL:CH09d_ECIII-L:...,210,43,...,0.078,63.8,12.4,117,APGKKVLYLGAASGTSVSHVSDVVGPEGVVYAVEFSHRPGRELISM...,~~~~~vldiG~G~G~~~~--------~~~~~g~d~~~~~~~~~~~~...,~~~~~VLDiG~G~G~~~~~la~~~~~~~~v~gvD~s~~~i~~a~~~...,TRESIVLDAGCGNGRSFL--------VPCMVGMDYCLGLLNDARAA...,4567999999999964335432,.++.+|||+|||+|.....+..
3,4,NOP1,NOP1 YDL014W,327,166,300,KMV65957,KMV65957 pep supercontig:ECIIIL:CH06_ECIII-L:7...,164,23,...,0.050,76.0,10.4,163,PGKKVLYLGAASGTSVSHVSDVVGPEGVVYAVEFSHRPGRELISMA...,~~~~vld~gcG~G~~~~~l~~~----~~v~~~D~~~~~~~~~~~--...,~~~~VLDiG~G~G~~~~~la~~~~~~~~v~gvD~s~~~i~~a~~~~...,EMKIVLDLGTSTGVITEQLRKR----NTVVSTDLNIRALESHRG--...,4678999999999998877655531,++.+|||+|||+|.+...+...+..
4,5,NOP1,NOP1 YDL014W,327,163,273,KMV65189,KMV65189 pep supercontig:ECIIIL:CH10_ECIII-L:2...,283,46,...,0.177,71.5,11.4,44,FIAPGKKVLYLGAASGTSVSHVSDVVGPEGVVYAVEFSHRPGRELI...,~~~~~~~vldiG~G~G~~~~~~~~~~--~~~v~~~d~~~~~~~~~~...,~~~~~~~VLDiG~G~G~~~~~la~~~~~~~~v~gvD~s~~~i~~a~...,YTKRGDSVLDLGCGKGGDLLKYERAG--IGEYYGVDIAEVSINDAR...,3456789999999999988777655198876543,...++.+|||+|||.|.++..+....|++...+.


The top hit with the score value much larger than the others contains the alignment in which we are interested. And so we can assign the aligned sequences from there as the ones we'll use.

In [14]:
aligned_query_seq = hh_df['qseq'][0]
aligned_hit_seq = hh_df['hseq'][0]

Because the aligned sequences may not start at position #1, that needs to be accounted for to get be able to use the numbers from the structure relative the query sequence. For the aligned hit, the position possibly not being the first residue needs to be accounted for to get accurate numbering for the positions of the equivalents.

In [15]:
start_pos_query_seq = hh_df['qstart'][0]
start_pos_hit_seq = hh_df['hstart'][0]

To verify things look good, we can see some of the initial alignment by running the next cell to print a slice of each and say what position it starts at:

In [16]:
limit = 60
print(aligned_query_seq[:limit])
print(aligned_hit_seq[:limit])
print(f"The top sequence starts at position {start_pos_query_seq}.")
print(f"The bottom sequence starts at position {start_pos_hit_seq}.")

ARGGAKVVIEPH-RHAGVYIARGKEDLLVTKNMAPGESVYGEKRISVEEPSKEDGVPPTK
AGLDRKVLVEPHPRFPGVYISRGKEDLLLTRNLVPGVSVYGEKRVAVDLEG-------MK
The top sequence starts at position 80.
The bottom sequence starts at position 41.


Note the aligned sequences and information on start position, and so things look good to actually do the report generation

## Setting report options

There are some **OPTIONAL** settings for the reports. You can choose not to define these and the default settings within the script will be used. For example, if you delete the next cell, the script will still run and represenative context details for the residue will be shown for residues with equivalents and not displayed for those without equivalents.

In [17]:
details_for_those_with_equivalents = True #default in script is that details of context will be shown; can override here
details_for_those_without_equivalents = False #default in script is that details of context will NOT be shown; can override here

## Main analysis step and report generating

Now because above we already defined the variables and dataframe containing the data we want to analyze, the script the next cell runs knows to use all those elements to see if the residues involved in the interaction of chain#1 with chain#2 in the specified structure have equivalents in the ortholog protein from the other organism that isn't baker's yeast.

In [18]:
%run -i make_report_on_equivalents_of_interacting_residues.py

Parsing data from PDBsum to identify chain A residues interacting with chain B residues ...

Identify any residues that are not in the span that is aligned ...

Checking for equivalents of the interacting residues in 
in the other aligned sequence ...

Determination of EQUIVALENTS Completed.

*************************************RESULTS************************************
The following percent of chain A interacting with chain B residues
have equivalents in the other aligned sequence:
100.0%
--------------------------------------------------------------------------------
Percent breakdown of the conservation categories for the interacting
residues of chain A that have equivalents in
the other aligned sequence:
identical: 53.1%
strongly conserved: 25.0%
weakly conserved: 6.2%
not conserved: 15.6%
--------------------------------------------------------------------------------
The specific residue numbers of the conservation categories for the interacting
residues of chain A that have eq

-----

### Making the reports more human readable

Note the report that is generated can be made more informative after-the-fact by running the script again and capturing all the output and then using the string `.replace()` method to change the chain designations that come from the PDBsum data to be the protein names and at the same time swap the 'other aligned sequence' for a better phrase to reference the sequence that was aligned to the same chain 1 in the structure. The idea is that the PDBsum data file doesn't have the protein names yet you can easily add them in after to make the report clearer. Plus, you probably have a better way to reference the 'other aligned sequence' than may be found in the HH-suite3-generated alignment. Adding a way to do this when calling the script would add more trouble than it is probably worth since you can perform this trick.    
An example is below in the next few cells. When you run the next one, you won't see any output; however, it will be captured.

In [19]:
%%capture out
%run -i make_report_on_equivalents_of_interacting_residues.py

This next cell will substitute in the protein names and text to replace the 'other aligned seqence', i.e., the 'hit' sequence from the alignment.

In [20]:
sys.stderr.write(out.stderr.replace("chain A","fibrillarin (Nop1p)").replace("chain B","Nop56p").replace("other aligned sequence","candidate EC ortholog of fibrillarin"))

Parsing data from PDBsum to identify fibrillarin (Nop1p) residues interacting with Nop56p residues ...

Identify any residues that are not in the span that is aligned ...

Checking for equivalents of the interacting residues in 
in the candidate EC ortholog of fibrillarin ...

Determination of EQUIVALENTS Completed.

*************************************RESULTS************************************
The following percent of fibrillarin (Nop1p) interacting with Nop56p residues
have equivalents in the candidate EC ortholog of fibrillarin:
100.0%
--------------------------------------------------------------------------------
Percent breakdown of the conservation categories for the interacting
residues of fibrillarin (Nop1p) that have equivalents in
the candidate EC ortholog of fibrillarin:
identical: 53.1%
strongly conserved: 25.0%
weakly conserved: 6.2%
not conserved: 15.6%
--------------------------------------------------------------------------------
The specific residue numbers of the 


You'd substitute the details of your chains of interest and information about the aligned sequence if you were modifying this notebook for your own use and re-running it.

----

Continue on with the next notebook in the series, [????????](???????). That notebook builds on the ground work here to demonstrate how to use snakemake to scale up this process to produce similar reports on the maintenance or loss of residues equivalent to those involved in interactions with a pair or designated proteins from a structure where the sequence in that structure has been used to identify a remote homolog using HH-sute3 software.
 
----